# Авторы
Студенты группы M3439:

- Тепляков Валерий
- Плешаков Алексей
- Филипчик Андрей

# Source code
Исходный код можно посмотреть [тут](https://github.com/Mervap/OptimizationMethods/tree/master/lab2)

In [1]:
import math

# Задание 2

In [2]:
import numpy as np

from lab1.task1.golden_ratio import GoldenRatio
from lab1.watcher import Watcher
from lab2.newton import Newton

f1 = (
    lambda x, y: 100 * (y - x) ** 2 + (1 - x) ** 2,       # function
    lambda x, y: [202 * x - 200 * y - 2, 200 * (y - x)],  # gradient
    lambda x, y: [[202, -200], [-200, 200]]               # hessian
)

f2 = (
    lambda x, y: 100 * (y - x ** 2) ** 2 + (1 - x) ** 2,                          # function
    lambda x, y: [2 * (200 * x ** 3 - 200 * x * y + x - 1), 200 * (y - x ** 2)],  # gradient
    lambda x, y: [[1200 * x ** 2 - 400 * y + 2, -400 * x], [-400 * x, 200]],      # hessian
)

f3_exp_e1 = lambda x, y: math.exp(-(x - 2) ** 2 / 9 - (y - 3) ** 2 / 4)
f3_exp_e2 = lambda x, y: math.exp(-(x - 1) ** 2 / 4 - (y - 1) ** 2)

f3 = (
    lambda x, y: (-2 * math.exp(-((x - 1) / 2) ** 2 - (y - 1) ** 2) +
                  -3 * math.exp(-((x - 2) / 3) ** 2 - ((y - 3) / 2) ** 2)),
    lambda x, y: [
        2 / 3 * (x - 2) * f3_exp_e1(x, y) + (x - 1) * f3_exp_e2(x, y),     # df/dx
        3 / 2 * (y - 3) * f3_exp_e1(x, y) + 4 * (y - 1) * f3_exp_e2(x, y), # df/dy
    ],
    lambda x, y: [
        [
            (-4 / 27 * (x - 2) ** 2 + 2 / 3) * f3_exp_e1(x, y) + (1 - 0.5 * (x - 1) ** 2) * f3_exp_e2(x, y), # df/dx^2
            -1 / 3 * (x - 2) * (y - 3) * f3_exp_e1(x, y) - 2 * (x - 1) * (y - 1) * f3_exp_e2(x, y),          # df/dxdy
        ],
        [
            -1 / 3 * (x - 2) * (y - 3) * f3_exp_e1(x, y) - 2 * (x - 1) * (y - 1) * f3_exp_e2(x, y),          # df/dydx
            (-3 / 4 * (y - 3) ** 2 + 3 / 2) * f3_exp_e1(x, y) + (4 - 8 * (y - 1) ** 2) * f3_exp_e2(x, y),    # df/dy^2
        ]
    ]
)

fs = [f1, f2, f3]

In [3]:
initial_points = [
    np.array([0, 1]),
    np.array([2, 2]),
    np.array([-1.0, 1.0]),
    np.array([3.0, 2.0]),
    np.array([0.1, -0.5]),
]

ind = 0
for (f, grad, hessian) in fs:
    f = Watcher(f, grad, hessian)
    opt = Newton(f, 1e-6, GoldenRatio)

    ind += 1
    print(ind)
    for ip in initial_points:
        try:
            mn = opt.opt(ip)
            print(mn, opt.get_iterations_cnt())
        except RuntimeError as e:
            if hasattr(e, 'message'):
                print(e.message)
            else:
                print(e)

1
[0.99999996 1.        ] 1
[1.00000004 1.00000004] 3
[0.99999992 1.        ] 5
[1.00000008 1.00000004] 7
[0.99999996 0.99999994] 9
2
[0.99999997 0.99999993] 4
[1.00000005 1.00000009] 19
[0.99999999 0.99999997] 39
[1.00000001 1.00000002] 60
[1. 1.] 75
3
[1.26303502 1.33439596] 4
[1.26303476 1.3343955 ] 9
[1.26303502 1.33439596] 13
[1.96715155 2.88611509] 18
[1.26303502 1.33439596] 24
